<a href="https://colab.research.google.com/github/rolandoteniya/Volatility-Forecast-EGARCH-LSTM/blob/main/stock_market_volatility_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#DOWNLOAD LIBRARY
!pip install -q arch
!pip install -q shap

#SET UP ENVIRONMENT
#libraries to to start with
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model
import tensorflow as tf
import shap
import warnings
warnings.filterwarnings('ignore')
from datetime import date
import math
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error


#ACQUIRE CORE & EXOGENOUS DATA
#downloading s&p500 data from 1 jan 2000 to present day
sp500 = yf.download(tickers="^GSPC", start="1999-12-02", end="2025-07-01")

#downloadiing vix data from 1 jan 2000 to present day
vix = yf.download(tickers="^VIX", start="1999-12-02", end="2025-07-01")

#PREPROCESS AND ALIGN DATA
#choose only close price for sp500
close_sp500 = sp500['Close']

#choose only close price for vix
close_vix = vix['Close']

#calculate log retuns using a vecotirse operation
sp500_logrtn = np.log(close_sp500 / close_sp500.shift(1))

#first value will be not a number so we remove it
sp500_logrtn = sp500_logrtn.dropna()

#augmented dickey-fuller (adf) test
"""1st test statistic
2nd p value
4th number of datapoints
"""
adfuller(sp500_logrtn)
#print(adfuller(sp500_logrtn))

#align the snp500 log rtns with the vix closing price
#inner join ensures that only dates present in both datasets are kept
aligned_data = pd.concat([sp500_logrtn, close_vix], axis=1, join='inner')

#rename columns for clarity
aligned_data.columns = ['sp500_log_retuns','close_vix']

#display first few rows of aligned data
#print(aligned_data.head())

#GENERATE TARGET VARIABLE
#calculate 21 day future realised volatility
future_volatility = sp500_logrtn.shift(-21).rolling(window=21).std()*(252**.5)

#first few values will be not a number so we remove it
future_volatility = future_volatility.dropna()
#align snp500 log rtns, vix and future realised volatility togehter
aligned_data = pd.concat([sp500_logrtn, close_vix, future_volatility], axis=1, join='inner')

#rename columns for clarity
aligned_data.columns = ['sp500 log rtns', 'close vix', 'future volatility']

#display first few rows of algined data
#print(aligned_data.head())

#SPLIT DATA
# train_data = []
# val_data = []
# test_data = []
# for i in range(0,int(len(aligned_data) * 0.7)):
#   train_data.append(aligned_data.iloc[i])
# # print(len(train_data))

# for i in range(int(len(aligned_data) * 0.7), int(len(aligned_data) * 0.85)):
#   val_data.append(aligned_data.iloc[i])
# # print(len(val_data))

# for i in range(int(len(aligned_data) * 0.85), len(aligned_data)):
#   test_data.append(aligned_data.iloc[i])
# # print(len(test_data))

# print(len(aligned_data))
# print(len(train_data)+ len(val_data) + len(test_data))

#first calculate the split points
split1 = int(len(aligned_data) * 0.7)
split2 = int(len(aligned_data) * 0.85)

#perform slicing in one step for each set
train_data = aligned_data.iloc[:split1]
val_data = aligned_data.iloc[split1:split2]
test_data = aligned_data.iloc[split2:]

# #verify splits
# print(len(train_data)+len(val_data)+len(test_data))
# print(len(aligned_data))

#TEST DIFFERENT GARCH PARAMETERS
# select log returns from training data
returns_train = train_data['sp500 log rtns']

#specify the GARCH(1,1) model
#we use p=1 and q=1 for the GARCH and ARCH terms.
garch_model = arch_model(returns_train, p=1, q=1)

#fit the model to the data
garch_results = garch_model.fit(disp="off")

#print the summary of the results
#print(garch_results.summary())

#FORECAST VOLATILITY USING GARCH(1,1) model that was trainded only on the training data.
garch_forecast = garch_results.forecast(horizon=len(val_data))

#get the forecasted variance. reset the index to remove the date.
#.values converts it to a simple numpy array
predicted_variance = garch_forecast.variance.iloc[0].values

#get the actual volatility from the validation set.
#resetting the index here isn't strictly necessary but is good practice
actual_volatility = val_data['future volatility'].values

#calculate the RMSE
#no need to align or fill, as both are now simple arrays of the same length
rmse_garch = np.sqrt(mean_squared_error(actual_volatility, np.sqrt(predicted_variance)))

print(f'GARCH(1,1) RMSE: {rmse_garch}')


egarch_model = arch_model(returns_train, vol="EGARCH",p=1, o=1, q=1)

#fit the model to the data
egarch_results = egarch_model.fit(disp='off')

#forecast volatility using egarch(1,1)
egarch_forecast = egarch_results.forecast(horizon=len(val_data), method="simulation")
predicted_variance = egarch_forecast.variance.iloc[0].values
actual_volatility = val_data['future volatility'].values
rmse_egarch = np.sqrt(mean_squared_error(actual_volatility, np.sqrt(predicted_variance)))

print(f'EGARCH(1,1) RMSE: {rmse_egarch}')

SyntaxError: invalid decimal literal (ipython-input-1-3533713889.py, line 130)